In [2]:
import threading
import time

### 传统方式

In [13]:
def coding():
    for x in range(3):
        print('正在进行coding...%s' %x)
        time.sleep(1)
        
def drawing():
    for x in range(3):
        print('正在进行drawing...%s' %x)
        time.sleep(1)
        
def main():
    coding()
    drawing()
    
if __name__ == '__main__':
    start = time.time()  # 返回当前时间的时间戳（1970纪元后经过的浮点秒数）。
    main()
    end = time.time()
    print('用时',end-start)

正在进行coding...0
正在进行coding...1
正在进行coding...2
正在进行drawing...0
正在进行drawing...1
正在进行drawing...2
用时 6.0022454261779785


### 多线程

In [12]:
def coding():
    for x in range(3):
        print('正在进行coding...%s' %x)
        time.sleep(1)
        
def drawing():
    for x in range(3):
        print('正在进行drawing...%s' %x)
        time.sleep(1)
        
def main():
    t1 = threading.Thread(target=coding)
    t2 = threading.Thread(target=drawing)
    
    t1.start()
    t2.start()
    
if __name__ == '__main__':
    start = time.time()
    main()
    end = time.time()
    print('用时',end-start)

正在进行coding...0
正在进行drawing...0
用时 0.005986213684082031
正在进行coding...1
正在进行drawing...1
正在进行coding...2
正在进行drawing...2


#### 查看当前进程当中有多少线程数

In [40]:
def coding():
    for x in range(3):
        print('正在进行coding...%s' %x)
        time.sleep(1)
        
def drawing():
    for x in range(3):
        print('正在进行drawing...%s' %x)
        time.sleep(1)
        
def main():
    t1 = threading.Thread(target=coding)
    t2 = threading.Thread(target=drawing)
    
    t1.start()
    t2.start()
    
if __name__ == '__main__':
    main()
    print(threading.enumerate())

正在进行coding...0
正在进行drawing...0
[<_MainThread(MainThread, started 48836)>, <Thread(Thread-4, started daemon 41924)>, <Heartbeat(Thread-5, started daemon 9208)>, <HistorySavingThread(IPythonHistorySavingThread, started 20780)>, <ParentPollerWindows(Thread-3, started daemon 22240)>, <Thread(Thread-32, started 40536)>, <Thread(Thread-33, started 6264)>]
正在进行coding...1
正在进行drawing...1
正在进行coding...2
正在进行drawing...2


#### 查看当前线程名称

In [41]:
def coding():
    for x in range(3):
        print('正在进行coding...%s' %threading.current_thread())
        time.sleep(1)
        
def drawing():
    for x in range(3):
        print('正在进行drawing...%s' %threading.current_thread())
        time.sleep(1)
        
def main():
    t1 = threading.Thread(target=coding)
    t2 = threading.Thread(target=drawing)
    
    t1.start()
    t2.start()
    
if __name__ == '__main__':
    main()

正在进行coding...<Thread(Thread-34, started 13120)>
正在进行drawing...<Thread(Thread-35, started 20076)>
正在进行coding...<Thread(Thread-34, started 13120)>
正在进行drawing...<Thread(Thread-35, started 20076)>
正在进行coding...<Thread(Thread-34, started 13120)>
正在进行drawing...<Thread(Thread-35, started 20076)>


In [43]:
class CodingThread(threading.Thread):
    # run是固定的
    def run(self):
        for x in range(3):
            print('正在进行coding...%s' %threading.current_thread())
            time.sleep(1)
        
class DrawingThread(threading.Thread):
    def run(self):
        for x in range(3):
            print('正在进行drawing...%s' %threading.current_thread())
            time.sleep(1)
        
def main():
    t1 = CodingThread()
    t2 = DrawingThread()
    
    t1.start()
    t2.start()
    
if __name__ == '__main__':
    main()   

正在进行coding...<CodingThread(Thread-36, started 14736)>
正在进行drawing...<DrawingThread(Thread-37, started 24392)>
正在进行coding...<CodingThread(Thread-36, started 14736)>
正在进行drawing...<DrawingThread(Thread-37, started 24392)>
正在进行coding...<CodingThread(Thread-36, started 14736)>
正在进行drawing...<DrawingThread(Thread-37, started 24392)>


### 多线程共享全局变量的问题

In [58]:
VALUE = 0

def add_value():
    global VALUE
    for x in range(1000000):
        VALUE += 1
    print('value:',VALUE)
        
def main():
    for x in range(2):
        t = threading.Thread(target=add_value)
        t.start()
        
if __name__ == '__main__':
    main() 

value: 1345566
value: 1398567


#### "上锁"

In [59]:
VALUE = 0

gLock = threading.Lock()

def add_value():
    global VALUE
    ##############
    gLock.acquire()
    for x in range(1000000):
        VALUE += 1
    gLock.release()
    ##############
    print('value:',VALUE)
        
def main():
    for x in range(2):
        t = threading.Thread(target=add_value)
        t.start()
        
if __name__ == '__main__':
    main() 

value: 1000000
value: 2000000


### Lock版本生产者和消费者

In [3]:
import random
import threading
import time

In [7]:
gMoney = 1000
gLock = threading.Lock()
gTotalTimes = 10
gTimes = 0

class Producer(threading.Thread):
    def run(self):
        global gMoney
        global gTimes
        while True:
            money = random.randint(100,1000)
            gLock.acquire()
            if gTimes >= 10:
                gLock.release()
                break
            gMoney += money
            gTimes += 1
            gLock.release()
            print('生产者%s生产了%d元钱，剩余%d元钱'%(threading.current_thread(), money, gMoney))
            time.sleep(0.5)
            
class Consumer(threading.Thread):
    def run(self):
        global gMoney
        while True:
            money = random.randint(100,1000)
            gLock.acquire()
            if gMoney >= money:
                gMoney -= money
                print('消费者%s消费了%d元钱，剩余%d元钱'%(threading.current_thread(), money, gMoney))
            else:
                if gTimes >= gTotalTimes:
                    gLock.release()
                    break
                print('%s消费者准备消费%d元钱，剩余%d元钱，不足！！！'%(threading.current_thread(), money, gMoney))
            gLock.release()
            time.sleep(0.5)
            
def main():
    for x in range(5):
        t = Consumer(name='消费者线程%d'%x)
        t.start()
        
    for x in range(5):
        t = Producer(name='生产者线程%d'%x)
        t.start()

        
if __name__ == '__main__':
    main()         

消费者<Consumer(消费者线程0, started 19148)>消费了601元钱，剩余399元钱
<Consumer(消费者线程1, started 13964)>消费者准备消费623元钱，剩余399元钱，不足！！！
消费者<Consumer(消费者线程2, started 39236)>消费了166元钱，剩余233元钱
<Consumer(消费者线程3, started 20636)>消费者准备消费874元钱，剩余233元钱，不足！！！
<Consumer(消费者线程4, started 46692)>消费者准备消费353元钱，剩余233元钱，不足！！！
生产者<Producer(生产者线程0, started 944)>生产了350元钱，剩余583元钱
生产者<Producer(生产者线程1, started 32844)>生产了163元钱，剩余746元钱
生产者<Producer(生产者线程2, started 32532)>生产了994元钱，剩余1740元钱
生产者<Producer(生产者线程3, started 15048)>生产了283元钱，剩余2023元钱
生产者<Producer(生产者线程4, started 22832)>生产了604元钱，剩余2627元钱
消费者<Consumer(消费者线程0, started 19148)>消费了971元钱，剩余1656元钱
消费者<Consumer(消费者线程1, started 13964)>消费了838元钱，剩余818元钱
消费者<Consumer(消费者线程2, started 39236)>消费了479元钱，剩余339元钱
<Consumer(消费者线程3, started 20636)>消费者准备消费994元钱，剩余339元钱，不足！！！
<Consumer(消费者线程4, started 46692)>消费者准备消费793元钱，剩余339元钱，不足！！！
生产者<Producer(生产者线程0, started 944)>生产了761元钱，剩余1100元钱
生产者<Producer(生产者线程1, started 32844)>生产了387元钱，剩余1487元钱
生产者<Producer(生产者线程2, started 32532)>生产了187元钱，剩余1674元钱
生产者<Prod

In [11]:
gMoney = 1000
gCondition = threading.Condition()
gTotalTimes = 10
gTimes = 0

class Producer(threading.Thread):
    def run(self):
        global gMoney
        global gTimes
        while True:
            money = random.randint(100,1000)
            gCondition.acquire()
            if gTimes >= gTotalTimes:
                gCondition.release()
                break
            gMoney += money
            gTimes += 1
            gCondition.notify_all()
            gCondition.release()
            print('生产者%s生产了%d元钱，剩余%d元钱'%(threading.current_thread(), money, gMoney))
            time.sleep(0.5)
            
class Consumer(threading.Thread):
    def run(self):
        global gMoney
        while True:
            money = random.randint(100,1000)
            gCondition.acquire()
            while gMoney <  money:
                if gTimes >= gTotalTimes:
                    gCondition.release()
                    return
                print('%s消费者准备消费%d元钱，剩余%d元钱，不足！！！'%(threading.current_thread(), money, gMoney))
                gCondition.wait()
            gMoney -= money
            print('消费者%s消费了%d元钱，剩余%d元钱'%(threading.current_thread(), money, gMoney))
            gCondition.release()
            time.sleep(0.5)
            
def main():
    for x in range(3):
        t = Consumer(name='消费者线程%d'%x)
        t.start()
        
    for x in range(5):
        t = Producer(name='生产者线程%d'%x)
        t.start()

        
if __name__ == '__main__':
    main()         

消费者<Consumer(消费者线程0, started 13012)>消费了703元钱，剩余297元钱
<Consumer(消费者线程1, started 20484)>消费者准备消费440元钱，剩余297元钱，不足！！！
<Consumer(消费者线程2, started 37272)>消费者准备消费353元钱，剩余297元钱，不足！！！
生产者<Producer(生产者线程0, started 22096)>生产了804元钱，剩余1101元钱
消费者<Consumer(消费者线程1, started 20484)>消费了440元钱，剩余661元钱
消费者<Consumer(消费者线程2, started 37272)>消费了353元钱，剩余308元钱
生产者<Producer(生产者线程1, started 10444)>生产了311元钱，剩余619元钱
生产者<Producer(生产者线程2, started 32340)>生产了385元钱，剩余1004元钱
生产者<Producer(生产者线程3, started 37236)>生产了182元钱，剩余1186元钱
生产者<Producer(生产者线程4, started 17752)>生产了504元钱，剩余1690元钱
消费者<Consumer(消费者线程0, started 13012)>消费了111元钱，剩余1579元钱
消费者<Consumer(消费者线程2, started 37272)>消费了804元钱，剩余775元钱
生产者<Producer(生产者线程0, started 22096)>生产了106元钱，剩余881元钱消费者<Consumer(消费者线程1, started 20484)>消费了570元钱，剩余311元钱

生产者<Producer(生产者线程1, started 10444)>生产了517元钱，剩余828元钱
生产者<Producer(生产者线程2, started 32340)>生产了475元钱，剩余1303元钱
生产者<Producer(生产者线程3, started 37236)>生产了487元钱，剩余1790元钱
生产者<Producer(生产者线程4, started 17752)>生产了385元钱，剩余2175元钱
消费者<Consumer(消费者线程0, sta